Simple ETL process for getting data from world bank

In [ ]:
# Defining required libraries and variables
import requests 
import pandas as pd

from pandas_gbq import to_gbq

project_id = 'complete-trees-452014-g4'
dataset_id = 'world_bank'
base_url = 'https://api.worldbank.org/v2/'

def get_wb_data(endpoint, params={}):
    url = base_url + endpoint
    parameters = params | {'format': 'json'} # Extend the parameters to keep response in json format

    response = requests.get(url, parameters)

    if response.status_code == 200:
        data = response.json()
    else:
        return response
    return data[1]

In [ ]:
# EXTRACT: getting countries data
countries = pd.DataFrame(get_wb_data(
    endpoint='country', 
    params={'per_page': '1000'}
))

# TRANSFORM: transforming column names
countries.columns = countries.columns.str.lower()

#LOAD: load the data into bigquery table
table_id = 'countries' # Table name
to_gbq(countries, f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')


In [96]:
# EXTRACT
indicators = pd.DataFrame(get_wb_data(
    'country/indicator/NY.GDP.MKTP.CD;FP.CPI.TOTL.ZG;SL.UEM.TOTL.ZS',
    {
        'source': '2',
        'date': '2000:2025',
        'per_page': '20000'
    }
))


In [97]:
# TRANSFORM
indicators['countryiso2code'] = indicators['country'].apply(lambda x: x['id'])

indicators_expanded = indicators['indicator'].apply(pd.Series)
indicators_expanded = indicators_expanded.add_prefix('indicator_')
indicators = pd.concat([indicators, indicators_expanded]).drop(columns=['indicator', 'country'])

# Getting important numbers
inflation = indicators[indicators['indicator_id'] == 'FP.CPI.TOTL.ZG']
gdp = indicators[indicators['indicator_id'] == 'NY.GDP.MKTP.CD']
unemployment_rate = indicators[indicators['indicator_id'] == 'SL.UEM.TOTL.ZS']



In [99]:

print(inflation.describe)

<bound method NDFrame.describe of       countryiso3code date  value unit obs_status  decimal scale  \
6384              NaN  NaN    NaN  NaN        NaN      NaN   NaN   
6385              NaN  NaN    NaN  NaN        NaN      NaN   NaN   
6386              NaN  NaN    NaN  NaN        NaN      NaN   NaN   
6387              NaN  NaN    NaN  NaN        NaN      NaN   NaN   
6388              NaN  NaN    NaN  NaN        NaN      NaN   NaN   
...               ...  ...    ...  ...        ...      ...   ...   
12763             NaN  NaN    NaN  NaN        NaN      NaN   NaN   
12764             NaN  NaN    NaN  NaN        NaN      NaN   NaN   
12765             NaN  NaN    NaN  NaN        NaN      NaN   NaN   
12766             NaN  NaN    NaN  NaN        NaN      NaN   NaN   
12767             NaN  NaN    NaN  NaN        NaN      NaN   NaN   

      countryiso2code    indicator_id                        indicator_value  
6384              NaN  FP.CPI.TOTL.ZG  Inflation, consumer prices (ann

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

population = pd.DataFrame(get_wb_data(
    'country/indicator/SP.POP.TOTL',
    {
        'per_page': '10000',
        'date': '2000:2025'
    }
))

# adding iso2code in the data frame
population['countryiso2code'] = population['country'].apply(lambda x: x['id'])

# removing uneccessary columns
population = population.drop(columns=['indicator', 'country'])

ph_population = population[population['countryiso2code'] == 'PH'].sort_values('date')

fig, ax = plt.subplots()
ax.plot(ph_population['date'], ph_population['value'],  marker='o', linestyle='-', linewidth=2, markersize=4, color='#1f77b4')

ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{int(x):,}'))

plt.xticks(rotation=45, fontsize=12, ha='right')  
plt.yticks(fontsize=12)

# Grid for readability
ax.grid(True, linestyle='--', alpha=0.6)

# # Add Title & Labels with Styling
ax.set_title('PH population 2000 - 2023', fontsize=14, fontweight='bold', color='#333')
ax.set_xlabel('Year', fontsize=12, fontweight='bold', color='#555', labelpad=0.4)
ax.set_ylabel('Population', fontsize=12, fontweight='bold', color='#555')



plt.show()